In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.tools import tool,ToolRuntime

@tool
def read_email(runtime):
    """Read an email from te given address"""
    #take email from state
    return runtime.state["email"]

@tool
def send_email(body):
    """Send an email to given address with the given subject and body"""
    #fake email sending
    return f"Email sent"


In [6]:
from langchain.agents import create_agent,AgentState
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

class EmailState(AgentState):
    email:str

agent=create_agent(
    model="gpt-5-nano",
    tools=[read_email,send_email],
    checkpointer=InMemorySaver(),
    state_schema=EmailState,
    middleware=[HumanInTheLoopMiddleware(
        interrupt_on={
            "read_email":False,
            "send_email":True,
        },
        description_prefix="Tool execution requires approval",
    ),
    ],
)

In [7]:
from langchain.messages import HumanMessage

config={"configurable":{"thread_id":"1"}}

response=agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [8]:
from pprint import pprint
pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Subject: '
                                                                          'Re: '
                                                                          'Meeting '
                                                                          'tomorrow\n'
                                                                          '\n'
                                                                          'Hi '
                                                                          'John,\n'
                                                                          '\n'
                                                                          'No '
                                                                          'problem—thanks '
                                                                          'for '
                                                                          'the '
     

In [9]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': "Subject: Re: Meeting tomorrow\n\nHi John,\n\nNo problem—thanks for the heads up. I'm flexible to reschedule. Would 10:00 AM or 3:00 PM tomorrow work for you? If neither time fits, please share a couple of alternatives and I'll adjust.\n\nBest regards,\nSeán"}, 'description': 'Tool execution requires approval\n\nTool: send_email\nArgs: {\'body\': "Subject: Re: Meeting tomorrow\\n\\nHi John,\\n\\nNo problem—thanks for the heads up. I\'m flexible to reschedule. Would 10:00 AM or 3:00 PM tomorrow work for you? If neither time fits, please share a couple of alternatives and I\'ll adjust.\\n\\nBest regards,\\nSeán"}'}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='fc1baa57ee21d8da0bc2aedebc29e630')]


In [10]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Subject: Re: Meeting tomorrow

Hi John,

No problem—thanks for the heads up. I'm flexible to reschedule. Would 10:00 AM or 3:00 PM tomorrow work for you? If neither time fits, please share a couple of alternatives and I'll adjust.

Best regards,
Seán


# Approve

In [11]:
from langgraph.types import Command
response=agent.invoke(
    Command(
        resume={"decisions":[{"type":"approve"}]}
    ),
    config=config# same threadid
)
pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='9a6db72b-8268-4f16-98e7-d7b42c98399e'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 858, 'prompt_tokens': 162, 'total_tokens': 1020, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 832, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsP6TcapTCN0BXyWYXCYIJxEl42Kv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b6e4f-3bd3-7ed0-9808-1f36818ce8a2-0', tool_calls=[{'name': 'read_email', 'args': {'runtime': 'defa

# Reject

In [13]:
response=agent.invoke(
    Command(
        resume={
            "decisions":[
                {
                    "type":"reject",
                    #explanation on rejection
                    "message":"No please sign up"
                }
            ]
        }
    ),
    config=config
)

In [14]:
pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='9a6db72b-8268-4f16-98e7-d7b42c98399e'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 858, 'prompt_tokens': 162, 'total_tokens': 1020, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 832, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsP6TcapTCN0BXyWYXCYIJxEl42Kv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b6e4f-3bd3-7ed0-9808-1f36818ce8a2-0', tool_calls=[{'name': 'read_email', 'args': {'runtime': 'defa

# Edit

In [17]:
response=agent.invoke(
    Command(
        resume={
            "decisions":[
                {
                    "type":"edit",
                    "edited_action":{
                        "name":"send_email",
                        "args":{"body":"you're hired"}
                    }
                }
            ]
        }
    ),
    config=config
)
pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='9a6db72b-8268-4f16-98e7-d7b42c98399e'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 858, 'prompt_tokens': 162, 'total_tokens': 1020, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 832, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CsP6TcapTCN0BXyWYXCYIJxEl42Kv', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b6e4f-3bd3-7ed0-9808-1f36818ce8a2-0', tool_calls=[{'name': 'read_email', 'args': {'runtime': 'defa